In [1]:
import struct

def left_rotate(value, shift):
    return ((value << shift) | (value >> (32 - shift))) & 0xFFFFFFFF

def md5(message):
    message = bytearray(message, 'utf-8')
    # Initialize variables
    a = 0x67452301
    b = 0xEFCDAB89
    c = 0x98BADCFE
    d = 0x10325476

    # Constants
    s = [7, 12, 17, 22] * 4 + [5, 9, 14, 20] * 4 + [4, 11, 16, 23] * 4 + [6, 10, 15, 21] * 4
    K = [0xD76AA478, 0xE8C7B756, 0x242070DB, 0xC1BDCEEE,
         0xF57C0FAF, 0x4787C62A, 0xA8304613, 0xFD469501,
         0x698098D8, 0x8B44F7AF, 0xFFFF5BB1, 0x895CD7BE,
         0x6B901122, 0xFD987193, 0xA679438E, 0x49B40821,
         0xF61E2562, 0xC040B340, 0x265E5A51, 0xE9B6C7AA,
         0xD62F105D, 0x02441453, 0xD8A1E681, 0xE7D3FBC8,
         0x21E1CDE6, 0xC33707D6, 0xF4D50D87, 0x455A14ED,
         0xA9E3E905, 0xFCEFA3F8, 0x676F02D9, 0x8D2A4C8A,
         0xFFFA3942, 0x8771F681, 0x6D9D6122, 0xFDE5380C,
         0xA4BEEA44, 0x4BDECFA9, 0xF6BB4B60, 0xBEBFBC70,
         0x289B7EC6, 0xEAA127FA, 0xD4EF3085, 0x04881D05,
         0xD9D4D039, 0xE6DB99E5, 0x1FA27CF8, 0xC4AC5665,
         0xF4292244, 0x432AFF97, 0xAB9423A7, 0xFC93A039,
         0x655B59C3, 0x8F0CCC92, 0xFFEFF47D, 0x85845DD1,
         0x6FA87E4F, 0xFE2CE6E0, 0xA3014314, 0x4E0811A1,
         0xF7537E82, 0xBD3AF235, 0x2AD7D2BB, 0xEB86D391]

    # Padding
    bit_len = len(message) * 8
    message.append(0x80)
    while len(message) % 64 != 56:
        message.append(0)
    message += struct.pack('<Q', bit_len)

    # Process each block
    for i in range(0, len(message), 64):
        chunk = message[i:i+64]

        a0, b0, c0, d0 = a, b, c, d

        for i in range(64):
            if i < 16:
                F = (b & c) | ((~b) & d)
                g = i
            elif i < 32:
                F = (d & b) | ((~d) & c)
                g = (5 * i + 1) % 16
            elif i < 48:
                F = b ^ c ^ d
                g = (3 * i + 5) % 16
            else:
                F = c ^ (b | (~d))
                g = (7 * i) % 16

            temp = d
            d = c
            c = b
            b = (b + left_rotate((a + F + K[i] + struct.unpack('<I', chunk[4*g:4*g+4])[0]) & 0xFFFFFFFF, s[i])) & 0xFFFFFFFF
            a = temp

        a = (a + a0) & 0xFFFFFFFF
        b = (b + b0) & 0xFFFFFFFF
        c = (c + c0) & 0xFFFFFFFF
        d = (d + d0) & 0xFFFFFFFF

    # Final hash
    result = struct.pack('<I', a) + struct.pack('<I', b) + struct.pack('<I', c) + struct.pack('<I', d)
    return result.hex()

# Example usage:
input_data = "Hello World"

md5_result = md5(input_data)
print(f"MD5 hash of '{input_data}': {md5_result}")

MD5 hash of 'Hello World': b10a8db164e0754105b7a99be72e3fe5


In [2]:
def encrypt_vigenere(plain_text, key):
    encrypted_text = ""
    for i in range(len(plain_text)):
        char = plain_text[i]
        if char.isalpha():
            shift = ord(key[i % len(key)]) - ord('A')
            if char.isupper():
                encrypted_char = chr((ord(char) + shift - ord('A')) % 26 + ord('A'))
                encrypted_text += encrypted_char 
    return encrypted_text

def decrypt_vigenere(encrypted_text, key):
    decrypted_text = ""
    for i in range(len(encrypted_text)):
        char = encrypted_text[i]
        if char.isalpha():
            shift = ord(key[i % len(key)]) - ord('A')
            if char.isupper():
                decrypted_char = chr((ord(char) - shift - ord('A')) % 26 + ord('A'))
                decrypted_text += decrypted_char
    return decrypted_text

plain_text = input("Enter Plain text: ")
key = input("Enter key: ")
plain_text = plain_text.upper()
key = key.upper()
encrypted_text = encrypt_vigenere(plain_text, key)
print("Encrypted text:", encrypted_text)
decrypted_text = decrypt_vigenere(encrypted_text, key)
print("Decrypted text:", decrypted_text)

Enter Plain text: Dwarkadas
Enter key: College
Encrypted text: FKLCOGHCG
Decrypted text: DWARKADAS


In [3]:
def encrypt_vernam(plain_text, key):
    ciphertext = ""
    if len(plain_text) == len(key):
        for i in range(len(plain_text)):
            conversion = (ord(plain_text[i]) - 65) ^ (ord(key[i]) - 65)
            conversion = conversion % 26
            ciphertext += chr(65 + conversion)
        return ciphertext
    else:
        return "Key and plaintext lengths are not equal"

plain_text = input("Enter plain text: ").upper()
key = input("Enter key: ").upper()

ciphertext = encrypt_vernam(plain_text, key)
print("Cipher text is:", ciphertext)

decrypted_text = encrypt_vernam(ciphertext, key) 
print("Decrypted text is:", decrypted_text)

Enter plain text: oak
Enter key: son
Cipher text is: COH
Decrypted text is: QAK


In [4]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import rsa

def generate_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(private_key, message):
    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

def verify_signature(public_key, message, signature):
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        print(f"Signature verification failed: {e}")
        return False

private_key, public_key = generate_key_pair()
message = b"Hello, this is a test message."
signature = sign_message(private_key, message)
print("Message:", message)
print("Signature:", signature.hex())
verification_result = verify_signature(public_key, message, signature)
print("Signature Verification Result:", verification_result)

Message: b'Hello, this is a test message.'
Signature: 3bf02267fdbc130e7f2dc50cde68c730563c36ae06d0fc918c70e3bff5ceeb8ce6473e4e17d3a0f1791a795b5d0cf0401c678469783256b0bcdb30e1f8644be36ecb52707686f85e09b11a449bb456375110c9ebfeddad66e84f1f695bfb128c1a0a09cdb678a335838fc8ac2232f5b4cf80769b3c4d0454e29ee7906f6bc24d7f2cf0403f65056234667f841b91c07b62929b032222cd804d89dc6ecf7cd208490555b2c7b574087729b71beae9a0014d4fa1091cce858e35e582ccdf34b5ffae41b55488dd9bfaa190625d47cc09424faffbf839357e5d5361e8d299ba406d3a6570f988c53769051ec5f5173badcc220416589e8283d8e805a463662b5561
Signature Verification Result: True
